In [9]:
import os
import sys
import yaml

#### Directory structure
<img src="tree.png" width="500" height="400">

#### Setting up config file(config.yml)
##### Placeholder

<pre>
 --> config_file['input_files']['models']: Path of model

 --> config_file['training_config']['max_trials']:  Number of hyperparamter fonfiguration(search space)
</pre>

In [10]:
with open('config.yaml') as f:
    config_file = yaml.full_load(f) 
    for item,doc in config_file.items():
        print(item,":",doc)

experiment_name : keras_tuner_fully_connected_pipeline
input_files : {'app': 'hyperparameter_tuning_custom_training.py', 'models': 'models/model_fc.py', 'dataset': 'datasets/data_loader_classification.py'}
training_config : {'epochs': 5, 'max_trials': 5}


### Adding conda environment to jupyter notebook
##### Step 1. Add snakemake environment to ipykernel




In [4]:
# %%bash

# conda env create -f environment.yml

# pip install --user ipykernel 

# python -m ipykernel install --user --name=snakemake


##### Step 2. Change the kernel to conda snakemake environment


### Running Snakemake to find best optimal hyperparameters

In [5]:
%%bash
snakemake --cores "all"

bash: /home/anwer/anaconda3/lib/libtinfo.so.6: no version information available (required by bash)
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 16
Rules claiming more threads will be scaled down.
Job stats:
job          count    min threads    max threads
---------  -------  -------------  -------------
run_model        1              1              1
total            1              1              1

Select jobs to execute...

[Thu Sep 22 09:12:16 2022]
rule run_model:
    input: hyperparameter_tuning_custom_training.py
    output: best_model/keras_tuner_fully_connected_pipeline/model_fc/best_param.txt
    jobid: 0
    reason: Missing output files: best_model/keras_tuner_fully_connected_pipeline/model_fc/best_param.txt
    resources: tmpdir=/tmp

2022-09-22 09:12:16.855116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-cr

GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
python hyperparameter_tuning_custom_training.py --epochs 5 --max_trials 5 --import_model_dataloader 'from models.model_fc import *' --best_model_path 'best_model/keras_tuner_fully_connected_pipeline/model_fc' --best_model_param 'best_model/keras_tuner_fully_connected_pipeline/model_fc/best_param.txt'


2022-09-22 09:12:18.560392: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-22 09:12:18.704101: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-22 09:12:19.372408: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/anwer/anaconda3/lib/
2022-09-22 09:12:19.372466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
4                 |?                 |num_layers
64                |?                 |units_1
0.0035801         |?                 |lr_1
tanh              |?                 |activation_1
False             |?                 |dropout_1
0.00010087        |?                 |lr

Epoch 1/5
4/4 [==============================] - 5s 499ms/step - loss: 21.8264 - mean_squared_error: 573.0854 - val_loss: 22.1945 - val_mean_squared_error: 571.9250
Epoch 2/5
4/4 [==============================] - 0s 51ms/step - loss: 21.7549 - mean_squared_error: 569.9575 - val_loss: 22.1205 - val_mean_squared_error: 568.6718
Epoch 3/5
4/4 [==============================] - 0s 51ms/step - loss: 21.6801 - mean_squared_error: 566.7476 - val_loss: 22.0430 - val_mean_squared_error: 565.2746
Epoch 4/5
4/4 [==============================] - 0s 47ms/step - loss: 21.6024 - mean_squared_error: 563.4139 - val_loss: 21.9643 - val_mean_squared_

[Thu Sep 22 09:12:50 2022]
Finished job 0.
1 of 1 steps (100%) done
Complete log: .snakemake/log/2022-09-22T091216.131096.snakemake.log


<img src="nvidia_gpu_consumption.png">

#### Optimal hyperparameters

In [24]:
path_hyperparameters = os.path.join("best_model",config_file['experiment_name'],config_file['input_files']['models'].split("/")[1].split(".")[0],"best_param.txt")
with open(path_hyperparameters, "r") as external_file:
    print(external_file.read())

{'num_layers': 3, 'units_1': 384, 'lr_1': 0.00027684469877965653, 'activation_1': 'tanh', 'dropout_1': True, 'lr': 0.009871011784920034, 'units_2': 32, 'lr_2': 0.0001, 'activation_2': 'relu', 'dropout_2': False, 'units_3': 32, 'lr_3': 0.0001, 'activation_3': 'relu', 'dropout_3': False, 'batch_size': 32}
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 13, 384)           768       
                                                                 
 dropout_2 (Dropout)         (None, 13, 384)           0         
                                                                 
 dense_5 (Dense)             (None, 13, 32)            12320     
                                                                 
 dense_6 (Dense)             (None, 13, 32)            1056      
                                                                 
 dense_7 (Den

#### Loading Best model

In [21]:
import tensorflow as tf
from tensorflow import keras


In [25]:
best_model_path = os.path.join("best_model",config_file['experiment_name'],config_file['input_files']['models'].split("/")[1].split(".")[0])
model = keras.models.load_model(best_model_path)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 13, 384)           768       
                                                                 
 dropout_2 (Dropout)         (None, 13, 384)           0         
                                                                 
 dense_5 (Dense)             (None, 13, 32)            12320     
                                                                 
 dense_6 (Dense)             (None, 13, 32)            1056      
                                                                 
 dense_7 (Dense)             (None, 13, 1)             33        
                                                                 
Total params: 14,177
Trainable params: 14,177
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.optimizer.get_config()

{'name': 'Adam',
 'learning_rate': 0.009871011784920034,
 'decay': 0.0,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

#### Train the model further 